In [2]:
import numpy as np
import math
from interval import imath
from interval import fpu
from interval import interval

In [14]:
def left(interval):
    return fpu.max(interval)[0]


def right(interval):
    return fpu.max(interval)[1]


def mid(interval):
    return (fpu.max(interval)[0] + fpu.max(interval)[1]) / 2


def norm(p_1, p_2):
    r = 0
    for i in range(len(p_1)):
        r += (p_1[i] - p_2[i]) ** 2
    return math.sqrt(r)

Напишем алгоритм, который будет находить экстремум функции вдоль заданного направления.
Алгоритм будет основываться на методе "Золотого сечения" так как является наиболее эффективным по стравнению с основными методами одномерной оптимизации.

In [33]:
def GoldenRatio(F, index, a, b, p, e_d, e_f):  #f(function), i(index of direction),
    # a(left border), b(right border), p(current point), e_d(error of d), e_f(error of f)
    phi = (1 + math.sqrt(5)) / 2  #constant of golden ratio
    x_1 = b - (b - a) / phi
    x_2 = a + (b - a) / phi
    p_1 = p.copy()  #current point
    p_2 = p.copy()  #current point
    p_1[index] = x_1
    p_2[index] = x_2
    f_1 = F(p_1)  #value in 1-st point
    f_2 = F(p_2)  #value in 2-nd point
    while (b - a > e_d) | (abs(f_1 - f_2) > e_f):  #termination criteria
        if f_1 <= f_2:
            b = x_2
            x_2 = x_1
            x_1 = b - (b - a) / phi

            p_1[index] = x_1
            p_2[index] = x_2

            f_2 = f_1
            f_1 = F(p_1)
        else:
            a = x_1
            x_1 = x_2
            x_2 = a + (b - a) / phi

            p_1[index] = x_1
            p_2[index] = x_2

            f_1 = f_2
            f_2 = F(p_2)

    best_point = []
    for i in range(len(p)):
        best_point.append((p_1[i] + p_2[i]) / 2)

    return best_point  #point of extremum with error e_d

Теперь реализуем алгоритм Moore-Skelboe

In [41]:
def MooreSkelboe(F, index, a, b, p, e_d, e_f):  #F(function), i(index of direction),
    # a(left border), b(right border), p(current point), e_d(error of d), e_f(error of f)
    interval_d = []
    for i in range(len(p)):
        interval_d.append(interval[p[i], p[i]])
    interval_d[index] = interval[a, b]

    interval_f = F(interval_d)
    set_of_intervals = [[interval_d, interval_f]]
    U = right(interval_f)
    w_f = len(interval_f)
    w_d = len(interval_d)
    best_interval = set_of_intervals[0]
    while (w_d > e_d) | (w_f > e_f):
        set_of_intervals.pop(0)
        mid_p = mid(best_interval[0][index])
        interval_1 = best_interval[0].copy()
        interval_2 = best_interval[0].copy()
        interval_1[index] = interval[left(best_interval[0][index]), mid_p]
        interval_1_f = F(interval_1)
        interval_2[index] = interval[mid_p, right(best_interval[0][index])]
        interval_2_f = F(interval_2)
        U = min(U, right(interval_1_f))
        U = min(U, right(interval_2_f))
        set_of_intervals.append([interval_1, interval_1_f])
        set_of_intervals.append([interval_2, interval_2_f])
        for el in set_of_intervals:
            if U < left(el[1]):
                set_of_intervals.remove(el)
        set_of_intervals.sort(key=lambda item: left(item[1]))
        best_interval = set_of_intervals[0]
        w_f = right(best_interval[1]) - left(best_interval[1])
        w_d = right(best_interval[0][index]) - left(best_interval[0][index])

    best_point = []
    for i in range(len(p)):
        best_point.append(mid(best_interval[0][i]))

    return best_point

Функция вычисляющая евклидово расстояние между 2 точками (2,p норму)

Функция принимающая функцию, множество определения и начальную точку.
Она раелизует метод наискорейшего спуска, последовательно ищет минимум вдоль направлений параллельных координатным осям и останавливается когда улучшение точности становится малым.

In [42]:
def FastSearch(F, D, p, e_d, e_f, method):  #F(function), D(set), p(start point), e(error)
    dimension = len(p)

    while True:
        p_0 = p
        for index in range(dimension):
            p = method(F, index, D[index][0], D[index][1], p, e_d, e_f)
        if norm(p_0, p) < e_d:
            break
        if F(p_0) - F(p) < e_f:
            break

    best_point = p
    best_value = F(p)

    return best_point, best_value

Рассмотрим пример работы функции на примере:

In [51]:
def F(x):
    return (x[0] ** 2) / 2 - (x[1] ** 2) / 4 + 3

In [52]:
p, v = FastSearch(F, [[-10, 10], [-10, 10]], [1, 1], 0.001, 0.001, GoldenRatio)  #point of minimum
print(p)
print(v)

[-2.163525435194824e-16, -9.999591436509917]
-21.997957224280615


Попробуем теперь применить тут алгоритм Moore-Skelboe для поиска минимума вдоль направления:

In [44]:
p, v = FastSearch(F, [[-10, 10], [-10, 10]], [1, 1], 0.001, 0.001, MooreSkelboe)  #point of minimum
print(p)
print(v)

[-0.00030517578125, -9.999923706054688]
-21.9996184851625


Приведем функции на которых можно протестировать работу алгоритмов

In [8]:
def print_result(F, D, p, e):
    p_1 = fast_search(F, [[-10, 10], [-10, 10]], [5, 5], 0.0001, 0.0001, MooreSkelboe)  #point of minimum
    y_1 = F(p_1)  #minimum value

    p_2 = fast_search(F, [[-10, 10], [-10, 10]], [5, 5], 0.0001, 0.0001, GoldenRatio)  #point of minimum
    y_2 = F(p_2)  #minimum value

    print("min of Moore-Skelboe:", y_1, p_1)
    print("min of Golden Ratio:", y_2, p_2)


In [9]:
print_result(F_1, [[-10, 10], [-10, 10]], [1, 1], 0.0001)

NameError: name 'F_1' is not defined

In [10]:
def F_1(x):
    return -20 * imath.exp(-0.2 * imath.sqrt(0.5 * (x[0] * x[0] + x[1] * x[1]))) - imath.exp(
        0.5 * (imath.cos(2 * math.pi * x[0]) + imath.cos(2 * math.pi * x[1]))) + math.exp(1) + 20


def F_2(x):
    return (1.5 - x[0] + x[0] * x[1]) * (1.5 - x[0] + x[0] * x[1]) + (2.25 - x[0] + x[0] * x[1] * x[1]) * (2.25 - x[0] + x[0] * x[1] * x[1]) + (
            2.625 - x[0] + x[0] * x[1] * x[1] * x[1]) * (2.625 - x[0] + x[0] * x[1] * x[1] * x[1])


def F_3(x):
    return (x[0] + 2 * x[1] - 7) ** 2 + (2 * x[0] + x[1] - 5) ** 2


def F_4(x):
    return 0.26 * (x[0] ** 2 + x[1] ** 2) - 0.48 * x[0] * x[1]

In [18]:
# print_result(F_4, [[-10, 10], [-10, 10]], [1, 1], 0.01)

p_1 = fast_search(F_2, [[-4, 4], [-4, 4]], [1, 1], 0.01, 0.01, Moore_Skelboe)  #point of minimum
y_1 = F_2(p_1)  #minimum value

# p_2 = fast_search(F_2, [[-4, 4], [-4,4]], [1, 1], 0.01, 0.01, argmin)  #point of minimum
# y_2 = F_2(p_2)  #minimum value

# print("min of Moore-Skelboe:", y_1, p_1)
print("min of Golden Ratio:", y_2, p_2)






KeyboardInterrupt: 

In [ ]:
def Borders(p, gradient, D):
    max_t = math.inf
    min_t = -math.inf
    dir = gradient / np.linalg.norm(gradient)
    for i in range(len(p)):
        if dir[i] > 0:
            max_t = min((D[i][1] - p[i]) / dir[i], max_t)
            min_t = max((D[i][0] - p[i]) / dir[i], min_t)
        elif gradient[i] < 0:
            min_t = max((D[i][1] - p[i]) / dir[i], min_t)
            max_t = min((D[i][0] - p[i]) / dir[i], max_t)

    borders = [p + dir * min_t, p + dir * max_t]
    return borders